# Example notebook

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import anatomize as an
import spatialdata as sd
from zarr.errors import PathNotFoundError
from spatialdata.models import X, Y

With myst it is possible to link in the text cell of a notebook such as this one the documentation of a function or a class.

Let's take as an example the function {func}`anatomize.exposure.adjust_gamma`. 
You can see that by clicking on the text, the link redirects to the API documentation of the function. 
Check the raw markdown of this cell to understand how this is specified.

This works also for any package listed by `intersphinx`. Go to `docs/conf.py` and look for the `intersphinx_mapping` variable. 
There, you will see a list of packages (that this package is dependent on) for which this functionality is supported. 

For instance, we can link to the class {class}`anndata.AnnData`, to the attribute {attr}`anndata.AnnData.obs` or the method {meth}`anndata.AnnData.write`.

Again, check the raw markdown of this cell to see how each of these links are specified.

You can read more about this in the [intersphinx page](https://www.sphinx-doc.org/en/master/usage/extensions/intersphinx.html) and the [myst page](https://myst-parser.readthedocs.io/en/v0.15.1/syntax/syntax.html#roles-an-in-line-extension-point).

In [3]:
try:
    blobs_sdata = sd.read_zarr("blobs.zarr")
except PathNotFoundError:
    blobs_sdata = sd.datasets.blobs(length=2048, n_channels=100)
    blobs_sdata.write("blobs.zarr")
blobs_sdata = sd.read_zarr("blobs.zarr")

/Users/srivarra/Davis Lab/anatomize/.venv/lib/python3.12/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/srivarra/Davis Lab/anatomize/.venv/lib/python3.12/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


In [5]:
blobs_image = blobs_sdata.images["blobs_image"]

In [6]:
blobs_image = blobs_image.chunk(chunks={"c": 1, "y": 2048, "x": 2048})

In [7]:
a, b = blobs_image.quantile(q=(0.1, 0.9), dim=[Y, X]).compute()

In [26]:
c, y, x = blobs_image.chunks

In [ ]:
an.exposure.adjust_gamma(blobs_image, gamma=1.2, gain=2.0).compute()

In [ ]:
an.exposure.basic.is_low_contrast(image=blobs_image, fraction_threshold=0.65, q=(0.1, 0.9)).compute()

In [6]:
import boost_histogram.axis as bha

In [7]:
c_starts = blobs_image.min(dim=[Y, X]).compute()
c_stops = blobs_image.max(dim=[Y, X]).compute()

In [8]:
bins = [
    bha.Regular(bins=256, start=c_start, stop=c_stop)
    for c_start, c_stop in zip(c_starts.to_numpy(), c_stops.to_numpy(), strict=True)
]

In [8]:
q = an.exposure.histogram(blobs_image, bins=256)

In [ ]:
q.compute()